# MSE and Diebold Mariano

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import yfinance as yf
import os
from sklearn.metrics import mean_squared_error
import collections

In [2]:
symbols = ['^GSPC', 'GOOG', 'MSFT']
symbols = ['^AEX', '^BFX', '^GDAXI']
start_date = '2010-01-01'
start_date = '2000-01-01'
end_date = '2022-04-08'
vPrices = yf.download(symbols, start=start_date, end=end_date)['Close'].dropna()
vReturns = vPrices.pct_change().dropna()
vReturns = np.log(vPrices / vPrices.shift(1)).dropna() *100
vNpReturns = np.array(vReturns.transpose())
vTrain = vNpReturns[:,:int(len(vNpReturns[0,:]) * 0.8)]
vTest = vNpReturns[:,int(len(vNpReturns[0,:]) * 0.8):]

[*********************100%***********************]  3 of 3 completed


In [3]:

# Specify the directory path where your NumPy files are located
directory = r'C:\Users\onim-\Desktop\Thesis\Forecasts Numpy'

# Get a list of all files in the directory
file_list = os.listdir(directory)

# Filter the list to include only NumPy files (files with .npy extension)
npy_files = [file for file in file_list if file.endswith('.npy')]

# Load the NumPy files and assign them to variables with corresponding names
loaded_data = {}
variableNames = [] 
for file in npy_files:
    file_path = os.path.join(directory, file)
    data = np.load(file_path)
    variable_name = os.path.splitext(file)[0]  # Use file title as the variable name
    variableNames.append(variable_name)
    loaded_data[variable_name] = data

def fnMakeSigma(mPredLSTM):
    mSigma_LSTM = np.zeros(loaded_data[variableNames[5]].shape)
    mData = mPredLSTM.transpose()
    mSigma_LSTM[0,0,:] = mData[0,:]
    mSigma_LSTM[1,1,:] = mData[1,:]
    mSigma_LSTM[2,2,:] = mData[2,:]
    mSigma_LSTM[0,1,:] = mData[3,:]
    mSigma_LSTM[1,0,:] = mData[3,:]
    mSigma_LSTM[0,2,:] = mData[4,:]
    mSigma_LSTM[2,0,:] = mData[4,:]
    mSigma_LSTM[1,2,:] = mData[5,:]
    mSigma_LSTM[2,1,:] = mData[5,:]
    return mSigma_LSTM

def fnMakeActual(mTest):
    mSigma = np.zeros((3,3,len(mTest[0,:])))
    for i in range(3):
        for j in range(3):
            mSigma[i,j,:] = mTest[i,:]* mTest[j,:]
    return mSigma
                

loaded_data[variableNames[0]] = fnMakeSigma(loaded_data[variableNames[0]])
loaded_data[variableNames[1]] = fnMakeSigma(loaded_data[variableNames[1]])
loaded_data[variableNames[2]] = fnMakeSigma(loaded_data[variableNames[2]])
loaded_data[variableNames[3]] = fnMakeSigma(loaded_data[variableNames[3]])

In [4]:
variableNames

['BatchLSTM',
 'BatchLSTM1',
 'LSTM_Bayes',
 'LSTM_pred',
 'mPred_ADCC_N',
 'mPred_ADCC_ST',
 'mPred_BEKK_N',
 'mPred_BEKK_ST',
 'mPred_CCC_Bayes_N',
 'mPred_CCC_Bayes_ST',
 'mPred_CCC_N',
 'mPred_CCC_ST',
 'mPred_Copula_N',
 'mPred_Copula_ST',
 'mPred_DCC_Bayes_N',
 'mPred_DCC_Bayes_ST',
 'mPred_DCC_N',
 'mPred_DCC_ST']

In [10]:
mActual = fnMakeActual(vTest)

# MSE

Here I make a large MSE matrix which is 12x12x1128 which is the number of variables as nrow and ncol and the depth is the time stamps $t$

In [6]:
lower_diagonal = mActual[:,:,0][np.tril_indices(mActual[:,:,0].shape[0], k=0), 0]

In [7]:
mActual[:,:,0]

array([[0.32638789, 0.12954058, 0.2323157 ],
       [0.12954058, 0.05141355, 0.09220413],
       [0.2323157 , 0.09220413, 0.1653572 ]])

In [8]:
lower_diagonal

array([[0.32638789, 0.12954058, 0.12954058, 0.2323157 , 0.2323157 ,
        0.2323157 ],
       [0.32638789, 0.32638789, 0.12954058, 0.32638789, 0.12954058,
        0.2323157 ]])

In [6]:
iT = len(vTest[0,:])
iN = len(variableNames)

In [7]:
mMSE_t = np.zeros((iN, iT ))
mMSE = np.zeros((iN, 1 ))

In [11]:
mActual.shape

(3, 3, 1128)

In [12]:
N=9
for t in range(iT):
    for i in range(iN):
        # mMSE_t[i, t] = 1/(9**2) * (loaded_data[variableNames[i]][:,:,t] - mActual[:,:,t] ).reshape(9,1).transpose() @ (loaded_data[variableNames[i]][:,:,t] - mActual[:,:,t] ).reshape(9,1) 
        mMSE_t[i, t] = 2/(N*(N+1)) * ((loaded_data[variableNames[i]][0,0,t]- mActual[0,0,t])**2 + (loaded_data[variableNames[i]][1,1,t]- mActual[1,1,t])**2 + (loaded_data[variableNames[i]][2,2,t]- mActual[2,2,t])**2 + (loaded_data[variableNames[i]][0,1,t]- mActual[0,1,t])**2 + (loaded_data[variableNames[i]][0,2,t]- mActual[0,2,t])**2 + (loaded_data[variableNames[i]][1,2,t]- mActual[1,2,t])**2)


In [13]:
mMSE = np.average(mMSE_t, axis=1).reshape(iN,1)

In [48]:
mMSE

array([[5.56666788],
       [5.70487746],
       [5.92049963],
       [5.91343452],
       [5.94452858],
       [5.92938   ],
       [6.016947  ],
       [6.03972335],
       [5.98837242],
       [6.01509737],
       [5.99290031],
       [6.02310974],
       [5.99290069],
       [6.0216909 ],
       [5.99522108],
       [6.01014796],
       [5.99860324],
       [6.01626938]])

In [37]:
from scipy.stats import t
def fnDM_Test(e1, e2, h=1, T=iT):    
    # Initialise lists
    e1_lst = e1
    e2_lst = e2
    d_lst  = e1-e2
    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    print(mean_d)
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

In [39]:
def fnNeweyWest(vE):
    V_hat = iT / (iN - 1) + np.sum(vE**2)
    V_hat1 = 0
    for i in range(iT):
        for j in range(iT):
            V_hat1 += (1- i/iT) * 2 * vE[i] * vE[j]
    V_hat1 = V_hat1 * (iT/(iT-1))
    vHat = V_hat + V_hat1

    omega = 1/(iT**3) * vHat
    return omega

In [40]:
from scipy.stats import t
def fnDM_Test_newey(e1, e2, h=1, T=iT):    
    # Initialise lists
    e1_lst = e1
    e2_lst = e2
    d_lst  = e1-e2
    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = fnNeweyWest(d_lst)#(gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    # harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    # DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

In [76]:
def newey_west(X, lag=1):
    n = len(X)
    X_mean = np.mean(X)
    S0 = np.mean((X - X_mean) ** 2)
    S1 = 0
    for i in range(lag):
        weight = 1.0 - (i / (lag + 1.0))
        S1 += weight * (np.mean((X[i+1:]-X_mean)*(X[:n-i-1]-X_mean)))
    return n * S0 + 2 * S1

def fnDM_Test1(e1, e2, h=1, T=None):
    if T is None:
        T = len(e1)

    # Initialize lists
    e1_lst = e1
    e2_lst = e2
    d_lst  = e1-e2
    
    # Mean of d        
    mean_d = np.mean(d_lst)
    
    # Use Newey-West to estimate HAC variance
    nw_var = newey_west(d_lst, h)
    
    # The test statistic is sqrt(T) times the average of d divided by its standard deviation
    DM_stat = np.sqrt(T) * mean_d / np.sqrt(nw_var)

    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt



In [35]:
from scipy.stats import ttest_rel
from scipy.stats import ttest_1samp

def diebold_mariano_test(errors_model1, errors_model2):
    """
    Perform the Diebold-Mariano test for comparing forecast accuracy of two models.
    
    errors_model1: array-like, forecast errors of model 1
    errors_model2: array-like, forecast errors of model 2
    
    Returns:
    dm_stat: float, test statistic
    p_value: float, p-value
    """
    diff = errors_model1 - errors_model2
    
    dm_stat, p_value = ttest_1samp(diff, 0)
    
    return dm_stat, p_value

# Example usage
errors_model1 = np.array([0.5, -0.2, 0.8, -0.1, 0.3])
errors_model2 = np.array([0.1, 0.3, -0.5, 0.6, -0.3])
dm_stat, p_value = diebold_mariano_test(errors_model1, errors_model2)
print("Diebold-Mariano test statistic:", dm_stat)
print("p-value:", p_value)

Diebold-Mariano test statistic: 0.5978790677185084
p-value: 0.582124488035517


In [60]:
dm_test

Note: you may need to restart the kernel to use updated packages.


In [17]:
mDiebold = np.zeros((iN,iN))

In [41]:
fnDM_Test_newey(mMSE_t[0,:], mMSE_t[4,:])

dm_return(DM=-11.268417578888036, p_value=5.529955721640863e-28)

In [33]:
for i in range(iN):
    for j in range(iN):
        if i>j:
            mDiebold[i,j]=fnDM_Test(mMSE_t[i,:], mMSE_t[j,:])[1]
        if j>i:
            mDiebold[i,j]=fnDM_Test(mMSE_t[i,:], mMSE_t[j,:])[0]



In [30]:
# TEST ONE 

for i in range(iN):
    for j in range(iN):
        if i>j:
            mDiebold[i,j]=diebold_mariano_test(mMSE_t[i,:], mMSE_t[j,:])[1]
        if j>i:
            mDiebold[i,j]=diebold_mariano_test(mMSE_t[i,:], mMSE_t[j,:])[0]

In [36]:
pd.DataFrame(data=mDiebold, columns=variableNames, index=variableNames).round(3) ####### test

,BatchLSTM,BatchLSTM1,LSTM_Bayes,LSTM_pred,mPred_ADCC_N,mPred_ADCC_ST,mPred_BEKK_N,mPred_BEKK_ST,mPred_CCC_Bayes_N,mPred_CCC_Bayes_ST,mPred_CCC_N,mPred_CCC_ST,mPred_Copula_N,mPred_Copula_ST,mPred_DCC_Bayes_N,mPred_DCC_Bayes_ST,mPred_DCC_N,mPred_DCC_ST
BatchLSTM,0.000,-0.273,-0.319,-0.307,-0.353,-0.318,-0.355,-0.360,-0.327,-0.343,-0.329,-0.346,-0.329,-0.342,-0.345,-0.347,-0.346,-0.350
BatchLSTM1,0.785,0.000,-0.345,-0.323,-0.405,-0.340,-0.399,-0.405,-0.355,-0.379,-0.358,-0.384,-0.358,-0.377,-0.384,-0.387,-0.385,-0.391
LSTM_Bayes,0.750,0.730,0.000,0.191,-0.197,-0.090,-0.547,-0.541,-0.352,-0.445,-0.364,-0.460,-0.364,-0.430,-0.495,-0.494,-0.500,-0.506
LSTM_pred,0.759,0.747,0.849,0.000,-0.255,-0.184,-0.631,-0.605,-0.413,-0.507,-0.424,-0.519,-0.424,-0.482,-0.594,-0.574,-0.596,-0.583
mPred_ADCC_N,0.724,0.686,0.844,0.799,0.000,0.189,-0.322,-0.347,-0.178,-0.265,-0.191,-0.282,-0.191,-0.264,-0.260,-0.284,-0.269,-0.299
mPred_ADCC_ST,0.751,0.734,0.928,0.854,0.851,0.000,-0.593,-0.560,-0.348,-0.453,-0.361,-0.467,-0.361,-0.429,-0.559,-0.527,-0.559,-0.537
mPred_BEKK_N,0.723,0.690,0.584,0.528,0.748,0.553,0.000,-0.454,1.116,0.043,0.770,-0.113,0.770,-0.069,0.701,0.762,0.738,0.042
mPred_BEKK_ST,0.719,0.685,0.588,0.545,0.729,0.576,0.650,0.000,1.639,2.369,1.879,2.214,1.879,0.885,0.551,0.662,0.552,0.651
mPred_CCC_Bayes_N,0.743,0.723,0.725,0.680,0.859,0.728,0.265,0.101,0.000,-1.199,-0.660,-1.034,-0.660,-0.712,-0.129,-1.050,-0.216,-1.738
mPred_CCC_Bayes_ST,0.732,0.705,0.656,0.612,0.791,0.651,0.966,0.018,0.231,0.000,1.409,-0.677,1.409,-0.250,0.273,0.135,0.248,-0.041


In [34]:
pd.DataFrame(data=mDiebold, columns=variableNames, index=variableNames).round(3)

,BatchLSTM,BatchLSTM1,LSTM_Bayes,LSTM_pred,mPred_ADCC_N,mPred_ADCC_ST,mPred_BEKK_N,mPred_BEKK_ST,mPred_CCC_Bayes_N,mPred_CCC_Bayes_ST,mPred_CCC_N,mPred_CCC_ST,mPred_Copula_N,mPred_Copula_ST,mPred_DCC_Bayes_N,mPred_DCC_Bayes_ST,mPred_DCC_N,mPred_DCC_ST
BatchLSTM,0.000,-0.273,-0.319,-0.307,-0.353,-0.318,-0.355,-0.360,-0.327,-0.343,-0.329,-0.346,-0.329,-0.342,-0.345,-0.347,-0.346,-0.350
BatchLSTM1,0.785,0.000,-0.345,-0.323,-0.405,-0.340,-0.399,-0.405,-0.355,-0.379,-0.358,-0.384,-0.358,-0.377,-0.384,-0.387,-0.385,-0.391
LSTM_Bayes,0.750,0.730,0.000,0.191,-0.197,-0.090,-0.547,-0.541,-0.352,-0.445,-0.364,-0.460,-0.364,-0.430,-0.495,-0.494,-0.500,-0.506
LSTM_pred,0.759,0.747,0.849,0.000,-0.255,-0.184,-0.631,-0.605,-0.413,-0.507,-0.424,-0.519,-0.424,-0.482,-0.594,-0.574,-0.596,-0.583
mPred_ADCC_N,0.724,0.686,0.844,0.799,0.000,0.189,-0.322,-0.347,-0.178,-0.265,-0.191,-0.282,-0.191,-0.264,-0.260,-0.284,-0.269,-0.299
mPred_ADCC_ST,0.751,0.734,0.928,0.854,0.851,0.000,-0.593,-0.560,-0.348,-0.453,-0.361,-0.467,-0.361,-0.429,-0.559,-0.527,-0.559,-0.537
mPred_BEKK_N,0.723,0.690,0.584,0.528,0.748,0.553,0.000,-0.454,1.116,0.043,0.770,-0.113,0.770,-0.069,0.701,0.762,0.738,0.042
mPred_BEKK_ST,0.719,0.685,0.588,0.545,0.729,0.576,0.650,0.000,1.639,2.369,1.879,2.214,1.879,0.885,0.551,0.662,0.552,0.651
mPred_CCC_Bayes_N,0.743,0.723,0.725,0.680,0.859,0.728,0.265,0.101,0.000,-1.199,-0.660,-1.034,-0.660,-0.712,-0.129,-1.050,-0.216,-1.738
mPred_CCC_Bayes_ST,0.732,0.705,0.656,0.612,0.791,0.651,0.966,0.018,0.231,0.000,1.409,-0.677,1.409,-0.250,0.273,0.135,0.248,-0.041


In [47]:
np.min(mDiebold)

-1.7377614081449257

In [48]:
print(pd.DataFrame(data=mDiebold, columns=variableNames, index=variableNames).round(3).to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrrr}
\toprule
{} &  LSTM\_Bayes &  LSTM\_pred &  mPred\_ADCC\_N &  mPred\_ADCC\_ST &  mPred\_BEKK\_N &  mPred\_BEKK\_ST &  mPred\_CCC\_Bayes\_N &  mPred\_CCC\_Bayes\_ST &  mPred\_CCC\_N &  mPred\_CCC\_ST &  mPred\_Copula\_N &  mPred\_Copula\_ST &  mPred\_DCC\_Bayes\_N &  mPred\_DCC\_Bayes\_ST &  mPred\_DCC\_N &  mPred\_DCC\_ST \\
\midrule
LSTM\_Bayes         &       0.000 &      0.191 &        -0.197 &         -0.090 &        -0.547 &         -0.541 &             -0.352 &              -0.445 &       -0.364 &        -0.460 &          -0.364 &           -0.430 &             -0.495 &              -0.494 &       -0.500 &        -0.506 \\
LSTM\_pred          &       0.849 &      0.000 &        -0.255 &         -0.184 &        -0.631 &         -0.605 &             -0.413 &              -0.507 &       -0.424 &        -0.519 &          -0.424 &           -0.482 &             -0.594 &              -0.574 &       -0.596 &        -0.583 \\
mPred\_ADCC\_N       &     

C:\Users\onim-\AppData\Local\Temp\ipykernel_9616\1051586758.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(data=mDiebold, columns=variableNames, index=variableNames).round(3).to_latex())


In [33]:
print(pd.DataFrame(data = mMSE, index=variableNames, columns=['MSE']).to_latex() )

\begin{tabular}{lr}
\toprule
{} &       MSE \\
\midrule
LSTM\_Bayes         &  5.920500 \\
LSTM\_pred          &  5.913435 \\
mPred\_ADCC\_N       &  5.944529 \\
mPred\_ADCC\_ST      &  5.929380 \\
mPred\_BEKK\_N       &  6.016947 \\
mPred\_BEKK\_ST      &  6.039723 \\
mPred\_CCC\_Bayes\_N  &  5.988372 \\
mPred\_CCC\_Bayes\_ST &  6.015097 \\
mPred\_CCC\_N        &  5.992900 \\
mPred\_CCC\_ST       &  6.023110 \\
mPred\_Copula\_N     &  5.992901 \\
mPred\_Copula\_ST    &  6.021691 \\
mPred\_DCC\_Bayes\_N  &  5.995221 \\
mPred\_DCC\_Bayes\_ST &  6.010148 \\
mPred\_DCC\_N        &  5.998603 \\
mPred\_DCC\_ST       &  6.016269 \\
\bottomrule
\end{tabular}



C:\Users\onim-\AppData\Local\Temp\ipykernel_9616\2779811357.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(data = mMSE, index=variableNames, columns=['MSE']).to_latex() )


In [34]:
N=9

In [35]:
 2/(N*(N+1))

True

# Logarithmic Scoring Rule Out-Of-Sample

In [14]:
def LSR_Normal(mSigt, vTest):
    mData = vTest
    iT = len(vTest[0,:])
    iN = 3
    vLL = np.zeros(iT)  
    for t in range(iT):
        vLL[t] = -0.5 * (iN*np.log(2*np.pi) + np.log(np.linalg.det(mSigt[:,:,t])) + mData[:,t].transpose() @ np.linalg.inv(mSigt[:,:,t]) @ mData[:,t])
    return np.sum(vLL) * -1


In [22]:
LSR_Normal(loaded_data[variableNames[1]], vTest)

C:\Users\onim-\AppData\Local\Temp\ipykernel_8152\2181901675.py:7: RuntimeWarning: invalid value encountered in log
  vLL[t] = -0.5 * (iN*np.log(2*np.pi) + np.log(np.linalg.det(mSigt[:,:,t])) + mData[:,t].transpose() @ np.linalg.inv(mSigt[:,:,t]) @ mData[:,t])


nan